In [1]:
import pandas as pd
import warnings
import os
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option("mode.chained_assignment", None)

In [80]:
path = "transfrom data/"
filepath = os.listdir(path)

In [84]:
df = pd.read_csv(path + "강남구.csv", sep="|")

In [97]:
default_df = pd.DataFrame(columns = list(df.columns))

In [98]:
for file in filepath:
    try:
        df = pd.read_csv(path + file, sep = "|")
    except UnicodeDecodeError:
        df = pd.read_csv(path + file, sep = "|", encoding="cp949")
    df = df.drop(["Unnamed: 7"], axis="columns")
    default_df = default_df.append(df)

In [99]:
default_df = default_df.drop_duplicates(["맛집명"], keep = "first")

In [104]:
default_df.to_excel(path + "Merged.xlsx", index=False)

In [101]:
default_df["영업시간"].value_counts()

매일 00:00 ~ 24:00                                     366
영업시간 매일 11:00 ~ 22:00                                269
매일 11:00 ~ 22:00                                     263
매일 11:30 ~ 22:00                                     171
영업시간 매일 11:30 ~ 22:00                                144
                                                    ... 
영업시간 금,토 16:00 ~ 07:00                                 1
월~토 06:00 ~ 19:00                                      1
영업시간 월~토 11:30 ~ 24:00 · 브레이크타임 월~토 15:00 ~ 17:30      1
영업시간 월,수,목,금,토,일 14:00 ~ 01:00                         1
금~일 12:00 ~ 24:00                                      1
Name: 영업시간, Length: 3003, dtype: int64

In [180]:
df = pd.read_excel(path + "Merged.xlsx")

In [181]:
for i in range(len(df)):
    text = df["영업시간"][i]
    try:
        if "영업시간" in text:
            df["영업시간"][i] = text.replace("영업시간", "")
    except: pass

In [84]:
import re
from tqdm import tqdm

In [191]:
df["영업일"] = ""
df["영업 시작시간"] = ""
df["영업 종료시간"] = ""
days = ["월", "화", "수", "목", "금", "토", "일"]
time_pattern = re.compile("\d{2}:\d{2} ~ \d{2}:\d{2}$")
for length in tqdm(range(len(df))):
    check_day = df["영업시간"][length]
    check_day = re.sub(r"[^가-힣]", "", str(check_day))
    open_days = []
    if "매일" in check_day:
        df["영업일"][length] = "매일"
    else:
        for day in days:
            if day in check_day:
                open_days.append(day)
        if "".join(open_days) == "월화수목금토일":
            df["영업일"][length] = "매일"
        else:
            df["영업일"][length] = ",".join(open_days)
    try:
        only_time = time_pattern.findall(df["영업시간"][length])[0]
        df["영업 시작시간"][length] = only_time.split()[0]
        df["영업 종료시간"][length] = only_time.split()[2]
    except:
        df["영업 시작시간"][length] = ""
        df["영업 종료시간"][length] = ""

100%|██████████| 11712/11712 [00:03<00:00, 3272.57it/s]


In [192]:
df.to_excel("load data/Splited_data.xlsx", index=False)

In [184]:
"""
for length in tqdm(range(len(df))):
    check_word = str(df["영업일"][length])
    if "," in check_word:
        only_day = check_word.split(",")
        day_append_lst = []
        for except_day in only_day:
            if len(except_day) > 2: # "매일"이 2글자이므로
                try:
                    remove_number_check = re.compile("\d{2}:\d{2}$")
                    remove_number = remove_number_check.findall(except_day)[0]
                    except_day = except_day.replace(remove_number, "")
                except: pass
                if len(except_day) > 2:
                    day_append_lst.append("")
                else:
                    day_append_lst.append(except_day)
            else:
                day_append_lst.append(except_day)

    elif "~" in check_word:
        only_day = check_word.split("~")
        day_append_lst = []
        for except_day in only_day:
            if len(except_day) > 2: # "매일"이 2글자이므로
                try:
                    remove_number_check = re.compile("\d{2}:\d{2}$")
                    remove_number = remove_number_check.findall(except_day)[0]
                    except_day = except_day.replace(remove_number, "")
                except: pass
                if len(except_day) > 2:
                    day_append_lst.append("")
                else:
                    day_append_lst.append(except_day)
            else:
                day_append_lst.append(except_day)
        df["영업일"][length] = ",".join(day_append_lst)
"""

'\nfor length in tqdm(range(len(df))):\n    check_word = str(df["영업일"][length])\n    if "," in check_word:\n        only_day = check_word.split(",")\n        day_append_lst = []\n        for except_day in only_day:\n            if len(except_day) > 2: # "매일"이 2글자이므로\n                try:\n                    remove_number_check = re.compile("\\d{2}:\\d{2}$")\n                    remove_number = remove_number_check.findall(except_day)[0]\n                    except_day = except_day.replace(remove_number, "")\n                except: pass\n                if len(except_day) > 2:\n                    day_append_lst.append("")\n                else:\n                    day_append_lst.append(except_day)\n            else:\n                day_append_lst.append(except_day)\n\n    elif "~" in check_word:\n        only_day = check_word.split("~")\n        day_append_lst = []\n        for except_day in only_day:\n            if len(except_day) > 2: # "매일"이 2글자이므로\n                try:\n       

In [1]:
import DataCollection
from tqdm import tqdm

완료한 구 '마포구' ,'서대문구','은평구','종로구',

이상한 구 '중구'

In [ ]:
gu_list = ['중구','용산구','성동구','광진구',
               '동대문구','성북구','강북구','도봉구','노원구','중랑구','강동구','송파구',
               '강남구','서초구','관악구','동작구','영등포구','금천구','구로구','양천구','강서구']
for gu_name in tqdm(gu_list):
    tmp = DataCollection.KakaoMapCollection(gu_name)
    tmp.DriverSettings(Turn_off_warning=True)
    tmp.StartCrawling()
    try:
        tmp.Loop()
    except Exception as e:
        print(e, gu_name)
    tmp.ExtractData()